In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
!pip install -q parlai
!pip install -q subword_nmt # extra requirement we need for this tutorial

# **Filter Data**

In [2]:
import pandas as pd
from termcolor import colored
import glob

def support_create_data():
  out = []
  path = "/content/drive/MyDrive/Khantil/Blender_Bot/data_2/filter_data1.csv"


  data  = pd.read_csv(path, names=["status", "msg", "time", "id"], header=None)
  # print(data)

  valid_conversation = 0
  invalid_conversation = 0

  for user_id_index in data["id"].unique():
    # print(user_id_index)
      session = [("", "")]
      sent_count = 0
      received_count = 0
  
      filtered_df = data[data["id"] == user_id_index]
      # print(filtered_df)
      last_flag = ""
      for i in range(len(filtered_df)):
          status = filtered_df.iloc[i, 0]
          # print(status)
          msg = filtered_df.iloc[i, 1]
          if status == "Sent" and str(msg) != "nan":
              sent_count += 1
              if last_flag != status and session[-1][0] != "":
                  if session[-1][1] != "":
                      session[-1][-1] = session[-1][-1] + "\n " + msg
                  else:
                      session[-1][-1] = msg

              last_flag = status

          elif status == "Received" and str(msg) != "nan":
              received_count += 1
              if last_flag != status:
                session.append([msg, ""])
              else:
                session[-1][0] = session[-1][0] + "\n " + msg
              last_flag = status

      if sent_count > 0 and received_count > 0:
          valid_conversation += 1
          out.append(session[1:])
      else:
          invalid_conversation += 1

  print(valid_conversation, invalid_conversation)  
  return out




In [7]:
out = support_create_data()

17689 23216


# **Create Data**

# **JD Function**

In [ ]:
# import pandas as pd
# from termcolor import colored
# import glob

# def support_create_data_jd():
#   out = []
#   path = "/content/drive/MyDrive/Khantil/Blender_Bot/data_2/dataset_jd.csv"


#   data  = pd.read_csv(path, names=["msg", "id", "time", "status", "tag"], header=None)

#   valid_conversation = 0
#   invalid_conversation = 0

#   for user_id_index in data["id"].unique():
#       session = [("", "")]
#       sent_count = 0
#       received_count = 0
  
#       filtered_df = data[data["id"] == user_id_index]
#       last_flag = ""
#       for i in range(len(filtered_df)):
#           status = filtered_df.iloc[i, 4]
#           msg = filtered_df.iloc[i, 0]
#           if status == "Sent" and str(msg) != "nan":
#               sent_count += 1
#               if last_flag != status and session[-1][0] != "":
#                   if session[-1][1] != "":
#                       session[-1][-1] = session[-1][-1] + "\n " + msg
#                   else:
#                       session[-1][-1] = msg

#               last_flag = status

#           elif status == "Received" and str(msg) != "nan":
#               received_count += 1
#               if last_flag != status:
#                 session.append([msg, ""])
#               else:
#                 session[-1][0] = session[-1][0] + "\n " + msg
#               last_flag = status

#       if sent_count > 0 and received_count > 0:
#           valid_conversation += 1
#           out.append(session[1:])
#       else:
#           invalid_conversation += 1

#   print(valid_conversation, invalid_conversation)  
#   return out


In [ ]:
# out = support_create_data_jd()

1764 880


In [3]:
from parlai.core.teachers import register_teacher, DialogTeacher
from parlai.scripts.display_data import DisplayData

@register_teacher("my_teacher")
class MyTeacher(DialogTeacher):
    def __init__(self, opt, shared=None):
        # opt is the command line arguments.
        
        # What is this shared thing?
        # We make many copies of a teacher, one-per-batchsize. Shared lets us store 
        
        # We just need to set the "datafile".  This is boilerplate, but differs in many teachers.
        # The "datafile" is the filename where we will load the data from. In this case, we'll set it to
        # the fold name (train/valid/test) + ".txt"
        opt['datafile'] = opt['datatype'].split(':')[0] + ".txt"
        super().__init__(opt, shared)
    
    def setup_data(self, datafile):
        # filename tells us where to load from.
        # We'll just use some hardcoded data, but show how you could read the filename here:
        print(f" ~~ Loading from {datafile} ~~ ")
        
        # setup_data should yield tuples of ((text, label), new_episode)
        # That is ((str, str), bool)
        
        # first episode
        # notice how we have call, response, and then True? The True indicates this is a first message
        # in a conversation
        # yield ('Hello', 'Hi'), True
        # # Next we have the second turn. This time, the last element is False, indicating we're still going
        # yield ('How are you', 'I am fine'), False
        # yield ("Let's say goodbye", 'Goodbye!'), False
        
        # # second episode. We need to have True again!
        # yield ("Hey", "hi there"), True
        # yield ("Deja vu?", "Deja vu!"), False
        # yield ("Last chance", "This is it"), False


        # yield ('jsk', 'jsk'), True
        # # Next we have the second turn. This time, the last element is False, indicating we're still going
        # yield ('kem cho', 'bas saru ho'), False
        # yield ("ha to madia", 'madia'), False

        # # second episode. We need to have True again!
        # yield ("jsk", "jsk"), True
        # yield ("shu hale", "bas jo hale rakhe"), False
        # yield ("to pachi call karu tamne", "ha vandho nai mota bhai"), False


        out = support_create_data()
        for b, kk in enumerate(out):
            for a, i in enumerate(kk):
                if i[0] != "" and i[1] != "":
                  if a == 0:
                    yield (i[0], i[1]), True
                  else:
                    yield (i[0], i[1]), False      


        
DisplayData.main(task="my_teacher")

12:08:33 | Opt:
12:08:33 |     allow_missing_init_opts: False
12:08:33 |     batchsize: 1
12:08:33 |     datapath: /usr/local/lib/python3.7/dist-packages/data
12:08:33 |     datatype: train:ordered
12:08:33 |     dict_class: None
12:08:33 |     display_add_fields: 
12:08:33 |     download_path: None
12:08:33 |     dynamic_batching: None
12:08:33 |     hide_labels: False
12:08:33 |     ignore_agent_reply: True
12:08:33 |     image_cropsize: 224
12:08:33 |     image_mode: raw
12:08:33 |     image_size: 256
12:08:33 |     init_model: None
12:08:33 |     init_opt: None
12:08:33 |     is_debug: False
12:08:33 |     loglevel: info
12:08:33 |     max_display_len: 1000
12:08:33 |     model: None
12:08:33 |     model_file: None
12:08:33 |     multitask_weights: [1]
12:08:33 |     mutators: None
12:08:33 |     num_examples: 10
12:08:33 |     override: "{'task': 'my_teacher'}"
12:08:33 |     parlai_home: /usr/local/lib/python3.7/dist-packages
12:08:33 |     starttime: Sep06_12-08
12:08:33 |     t

# **Train on new data**

In [4]:
import torch
torch.cuda.empty_cache()

In [5]:
from parlai.scripts.train_model import TrainModel

TrainModel.main(
    # similar to before
    task='my_teacher', 
    model='transformer/generator',
    model_file='/content/drive/MyDrive/Khantil/Blender_Bot/model_02_initial_zoo/test',


    
    # initialize with a pretrained model
    init_model='zoo:tutorial_transformer_generator/model',
    # init_model='/content/drive/MyDrive/Khantil/Blender_Bot/models_01/90m',
    


    # arguments we get from the pretrained model.
    # Unfortunately, these must be looked up separately for each model.
    n_heads=16, n_layers=8, n_positions=512, text_truncate=512,
    label_truncate=128, ffn_size=2048, embedding_size=512,
    activation='gelu', variant='xlm',
    dict_lower=True, dict_tokenizer='bpe',
    dict_file='zoo:tutorial_transformer_generator/model.dict',
    # dict_file='/content/drive/MyDrive/Khantil/Blender_Bot/models_01/90m.dict',
    learn_positional_embeddings=True,
    
    # some training arguments, specific to this fine-tuning
    # use a small learning rate with ADAM optimizer
    lr=1e-5, optimizer='adam',
    warmup_updates=100,
    # early stopping on perplexity
    validation_metric='ppl',
    # train at most 10 minutes, and validate every 0.25 epochs
    max_train_time=1000, validation_every_n_epochs=0.25,
    
    # depend on your gpu. If you have a V100, this is good
    batchsize=12, fp16=True, fp16_impl='mem_efficient',
    
    # speeds up validation
    skip_generation=True,
    
    # helps us cram more examples into our gpu at a time
    dynamic_batching='full',
)

12:09:17 | building data: /usr/local/lib/python3.7/dist-packages/data/models/tutorial_transformer_generator/tutorial_transformer_generator_v1.tar.gz
12:09:17 | Downloading http://parl.ai/downloads/_models/tutorial_transformer_generator/tutorial_transformer_generator_v1.tar.gz to /usr/local/lib/python3.7/dist-packages/data/models/tutorial_transformer_generator/tutorial_transformer_generator_v1.tar.gz


12:10:27 | building dictionary first...
12:10:27 | No model with opt yet at: /content/drive/MyDrive/Khantil/Blender_Bot/model_02_initial_zoo/test(.opt)
12:10:27 | your model is being loaded with opts that do not exist in the model you are initializing the weights with: allow_missing_init_opts: False,download_path: None,loglevel: info,dynamic_batching: full,verbose: False,is_debug: False,datapath: /usr/local/lib/python3.7/dist-packages/data,final_extra_opt: ,eval_dynamic_batching: None,num_workers: 0,max_train_steps: -1,log_every_n_steps: 50,validation_every_n_steps: -1,load_from_checkpoint: True,world_logs: ,save_format: conversations,log_keep_fields: all,tensorboard_logdir: None,wandb_log: False,wandb_name: None,wandb_project: None,wandb_entity: None,mutators: None,n_encoder_layers: -1,n_decoder_layers: -1,model_parallel: False,checkpoint_activations: False,beam_block_full_context: True,beam_length_penalty: 0.65,topk: 10,topp: 0.9,beam_delay: 30,beam_block_list_filename: None,temperat

({'exs': SumMetric(1.985e+04),
  'clen': AverageMetric(53.26),
  'ctrunc': AverageMetric(0.01169),
  'ctrunclen': AverageMetric(6.306),
  'llen': AverageMetric(56.95),
  'ltrunc': AverageMetric(0.1645),
  'ltrunclen': AverageMetric(13.27),
  'loss': AverageMetric(0.3388),
  'ppl': PPLMetric(1.403),
  'token_acc': AverageMetric(0.9202),
  'token_em': AverageMetric(0.5026),
  'exps': GlobalTimerMetric(515.5),
  'ltpb': GlobalAverageMetric(2324),
  'ltps': GlobalTimerMetric(2.252e+04),
  'ctpb': GlobalAverageMetric(2499),
  'ctps': GlobalTimerMetric(2.421e+04),
  'tpb': GlobalAverageMetric(4824),
  'tps': GlobalTimerMetric(4.672e+04),
  'lr': GlobalAverageMetric(9.9e-06),
  'gpu_mem': GlobalAverageMetric(0.0776),
  'total_train_updates': GlobalFixedMetric(1229)},
 {'exs': SumMetric(1.985e+04),
  'clen': AverageMetric(53.26),
  'ctrunc': AverageMetric(0.01169),
  'ctrunclen': AverageMetric(6.306),
  'llen': AverageMetric(56.95),
  'ltrunc': AverageMetric(0.1645),
  'ltrunclen': AverageMetr

# **Interactive testing**

In [7]:
from parlai.scripts.interactive import Interactive


Interactive.main(task='my_teacher',
    model_file='/content/drive/MyDrive/Khantil/Blender_Bot/model_02_initial_zoo/test')

12:35:39 | Using CUDA
12:35:39 | loading dictionary from /content/drive/MyDrive/Khantil/Blender_Bot/repeat_01/test.dict
12:35:39 | num words = 54944
12:35:40 | TransformerGenerator: full interactive mode on.
12:35:41 | Total parameters: 87,508,992 (87,508,992 trainable)
12:35:41 | Loading existing model params from /content/drive/MyDrive/Khantil/Blender_Bot/repeat_01/test
12:36:01 | Opt:
12:36:01 |     activation: gelu
12:36:01 |     adafactor_eps: '[1e-30, 0.001]'
12:36:01 |     adam_eps: 1e-08
12:36:01 |     add_p1_after_newln: False
12:36:01 |     aggregate_micro: False
12:36:01 |     allow_missing_init_opts: False
12:36:01 |     attention_dropout: 0.0
12:36:01 |     batchsize: 12
12:36:01 |     beam_block_full_context: True
12:36:01 |     beam_block_list_filename: None
12:36:01 |     beam_block_ngram: -1
12:36:01 |     beam_context_block_ngram: -1
12:36:01 |     beam_delay: 30
12:36:01 |     beam_length_penalty: 0.65
12:36:01 |     beam_min_length: 1
12:36:01 |     beam_size: 1
12:

KeyboardInterrupt: ignored